In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [190]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [191]:
%%capture pwd
!pwd

In [192]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('energy-transition-minerals-what-are-they-and-where-will-they-come-from',
 300,
 500)

# Fig 1

In [193]:
df = pd.read_excel("raw/Book1.xlsx")
df=df.set_index('Unnamed: 0').stack().reset_index()
df.columns=['metal','industry','value']

In [194]:
f = "fig1_minerals"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,metal,industry,value
0,Steel,Geothermal,3
1,Steel,Hydro,3
2,Steel,Nuclear,3
3,Steel,Bioenergy,3
4,Steel,Electricity networks,3


In [195]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "industry:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="top",
            labelAngle=-45,
            domain=False,
            offset=5,
            ticks=False
        ),
    )
)
dots = base.mark_point(color=colors["eco-mid-blue"],fill=colors["eco-light-blue"]).encode(
    y=alt.Y(
        "metal:N",
        sort=[],
        axis=alt.Axis(
            grid=True,
            title="",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            domain=False,
            ticks=False
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    ),
    size=alt.Size('value:Q',scale=alt.Scale(domain=[1, 5]),legend=None),
)
layer1 = (
    ((dots).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.Chart(...)

# Fig 2

In [196]:
df = pd.read_excel("raw/Book1.xlsx",sheet_name='Sheet2')
df=df.set_index(['Cat','Unnamed: 0']).stack().reset_index()
df.columns=['cat','industry','metal','value']

In [197]:
f = "fig2_technologies"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,cat,industry,metal,value
0,A,Electric car,Copper,53.0
1,A,Electric car,Nickel,40.0
2,A,Electric car,Cobalt,13.0
3,A,Electric car,Graphite,66.0
4,A,Electric car,Manganese,24.0


In [198]:
base = alt.Chart(f2).mark_bar(color=colors["eco-turquiose"],size=20,opacity=0.8).encode(
     x=alt.X(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            titleAlign="right",
            titleAnchor="end",
            title="Power generation (kg/MW)",
            titleY=-20,
            titleX=393,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            format='s'
        ),
    ),
    y=alt.Y(
        "industry:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleX=395,
            titleY=65,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="right",
            title='Transport (kg/vehicle)',
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-mid-blue"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color('metal:N',sort=[],legend=alt.Legend(title=''),scale=alt.Scale(range=['brown','#929292','royalBlue','black',
                                        'pink',colors['eco-turquiose'],'gold',colors['eco-dot'],colors['eco-orange'],colors['eco-mid-blue'],
                                                                                        colors['eco-green']])),
    # order='metal:N'
)
bars1=base.encode(
     x=alt.X(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            orient='top',
            title='',
            labelColor=colors['eco-mid-blue'],
            tickColor=colors["eco-mid-blue"],
            domainColor=colors["eco-mid-blue"],
        ),
    )).transform_filter("datum.cat=='A'")
bars2=base.transform_filter("datum.cat=='B'")
r = (
    alt.Chart(pd.DataFrame([{"value": 0, "industry": 'Offshore wind'},
                            {"value": 16000, "industry": 'Offshore wind'}, 
                           ]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-mid-blue"],yOffset=-20)
    .encode(x=alt.X("value:Q", sort=[]), y=alt.Y("industry:N",sort=[]))
)
layer1 = (
    ((bars1+(bars2+r))
     .resolve_scale(x='independent')
     .properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

In [782]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ days before  since infection ➡",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect on pass performance",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
points = line.mark_point(
    size=25, color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
)
area = base.mark_area(opacity=0.4, color=colors["eco-turquiose"]).encode(
    y="e1:Q", y2="e2:Q"
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "> 225", "y": 0}, {"x": "225 <", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
axis2 = (
    alt.Chart(pd.DataFrame([{"x": "75", "y": -0.16}, {"x": "75", "y": 0.05}]))
    .mark_line(strokeWidth=1, xOffset=25, color=colors["eco-dot"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
layer1 = (
    ((area + line + points + axis1 + axis2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)